#Shell

#Formula target:

#Exchange_Reg = (Listed) + License_Count + Incident Count + Compliance_Maturity + Country_Reg − BVI



In [1]:
# Early imports to ensure availability before any typing or utilities
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Resolve CSV paths relative to the notebook directory with fallbacks to parent dirs
def resolve_csv_path(filename: str) -> str:
	base = Path.cwd()
	candidates = [base / filename, base.parent / filename, base.parent.parent / filename]
	for p in candidates:
		if p.exists():
			return str(p)
	# default to first candidate; downstream will error clearly if truly missing
	return str(candidates[0])


In [2]:
# Early load to ensure original_df exists for downstream cells that reference it
ORIGINAL_CSV_PATH = resolve_csv_path('original_spreadsheet.csv')
try:
	original_df = pd.read_csv(ORIGINAL_CSV_PATH)
	if 'Crypto Exchange' in original_df.columns:
		original_df['Crypto Exchange'] = original_df['Crypto Exchange'].astype(str).str.strip()
except Exception as _e:
	print('Early load failed, will rely on later loader:', _e)


In [3]:
# Config and canonicalization helpers

# Canonical names for known regs/licenses to avoid double counting
LICENSE_CANON_MAP = {
	'mica': 'MiCA',
	'bitlicense': 'BitLicense',
	'fca': 'FCA',
	'fincen': 'FinCEN_MSB',
	'msb': 'FinCEN_MSB',
	'vasp': 'VASP',
	'fatf': 'FATF',
	'austrac': 'AUSTRAC',
	'fintrac': 'FINTRAC',
	'sec': 'SEC_CFTC',
	'cftc': 'SEC_CFTC',
}

# Severity keywords for incidents
INCIDENT_SEVERITY_WEIGHTS = [
	(r'criminal|fraud|felony', 3.0),
	(r'class\s*action|settlement|consent order', 2.5),
	(r'fine|penalt(y|ies)|sanction', 2.0),
	(r'warning|notice|advisory', 1.0),
]

# Product weights defaults/override (can be tuned later)
_DEFAULT_PRODUCT_WEIGHTS = {
	'Product_Options': 3.0,
	'Product_Futures': 3.0,
	'Product_Margin': 2.0,
	'Product_Staking': 1.0,
	'Product_P2P_OTC': 1.0,
	'Product_NFTs': 0.5,
	'Product_Launchpad': 1.0,
	'Product_Crypto_Loans': 2.0,
	'Product_Spot': 0.5,
}

if 'PRODUCT_WEIGHTS' not in globals():
	PRODUCT_WEIGHTS = _DEFAULT_PRODUCT_WEIGHTS.copy()
else:
	# Do not overwrite any pre-set weights; only fill missing keys
	PRODUCT_WEIGHTS = dict(PRODUCT_WEIGHTS)
	for k, v in _DEFAULT_PRODUCT_WEIGHTS.items():
		PRODUCT_WEIGHTS.setdefault(k, v)


In [4]:
# Data quality profiling cell: summarize coverage and inferred fields (robust)

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

# Ensure we have a dataframe to profile
try:
	_df_for_profile = original_df
except NameError:
	try:
		_df_for_profile = load_original_df()
	except Exception:
		_df_for_profile = pd.read_csv(ORIGINAL_CSV_PATH)

profile_df = profile_data(_df_for_profile)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [5]:
# Data quality profiling cell: summarize coverage and inferred fields

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

profile_df = profile_data(original_df)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [6]:
# Data quality profiling cell: summarize coverage and inferred fields

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

profile_df = profile_data(original_df)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [7]:
# Data quality profiling cell: summarize coverage and inferred fields

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

profile_df = profile_data(original_df)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [8]:
# Data quality profiling cell: summarize coverage and inferred fields

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

profile_df = profile_data(original_df)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [9]:
# Imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from typing import Tuple, Dict

pd.set_option('display.max_columns', 200)



In [10]:
# Column discovery utilities
import re as _reutil


def columns_with_prefix(df: pd.DataFrame, prefix: str) -> list:
	p = prefix.lower()
	return [c for c in df.columns if isinstance(c, str) and c.lower().startswith(p)]


def columns_matching(df: pd.DataFrame, pattern: str) -> list:
	regex = _reutil.compile(pattern, flags=_reutil.IGNORECASE)
	return [c for c in df.columns if isinstance(c, str) and bool(regex.search(c))]


def discover_incident_columns(df: pd.DataFrame) -> Tuple[list, list]:
	"""Return (text_like_cols, numeric_flag_cols) related to incidents/enforcement."""
	text_like = columns_matching(df, r'incident|enforc|fine|penalt|sanction|warning|notice|action')
	flags = [c for c in text_like if df[c].dtype != object]
	texts = [c for c in text_like if df[c].dtype == object]
	return texts, flags


def discover_product_explicit_cols(df: pd.DataFrame) -> list:
	return columns_with_prefix(df, 'Product_')



In [11]:
# Load original CSV (central source for processing)
ORIGINAL_CSV_PATH = resolve_csv_path('original_spreadsheet.csv')

def load_original_df(path: str = ORIGINAL_CSV_PATH) -> pd.DataFrame:
	df = pd.read_csv(path)
	# Normalize key columns
	if 'Crypto Exchange' in df.columns:
		df['Crypto Exchange'] = df['Crypto Exchange'].astype(str).str.strip()
	return df

# Load only if not already present
try:
	original_df
except NameError:
	original_df = load_original_df()
original_df.head(3)


,`,Crypto Exchange,Link to doc with full analysis,Exchange Size (Large/Medium/Small),Country/Region(s) of Operation,Ownership & Governance Structure,"Products Offered (spot, futures, options, etc.)",Product Launch Dates,Revenue/Profit by Region,"Regulatory Exposure (licenses, jurisdictions)","Key Regulatory Frameworks (MiCA, BitLicense, etc.)","Compliance Requirements (AML/KYC, disclosure, etc.)","Regulatory Incidents (fines, violations, etc.)",Data Sources Used,Progress Notes,Last Updated,Peer Reviewer
0,Colin,Binance,NaN,NaN,"Asia & Europe, restricted in US, UK, China",Privately owned,"Spot trading, margin, perpetual, options futur...","Binance Futures (Sep 2019), Options (Mar 2020)...",Undisclosed but believed to exceed $1B annually,"High global regulatory scrutiny, ongoing inves...","Affected by MiCA (EU), FATF AML rules, U.S. se...",KYC became mandatory in August 2021; Binance h...,"UK FCA bans, Dutch fines, SEC/CFTC lawsuits, D...","Reuters, CoinDesk, Sanction Scanner, Binance c...",NaN,NaN,NaN
1,Ethan,ByBit,Doc,Large,"Global: HQ in Dubai (VARA-regulated), expanded...",Founded March 2018 by Ben Zhou (ex-EMU Forex);...,Spot trading; perpetual futures (up to 100× le...,Perpetual futures launched March 2018; spot tr...,Non-public; estimated ~60% trading volume from...,VARA crypto license (Dubai); Cyprus VASP regis...,MiCAR (29 EEA states passporting); FATF Travel...,Tiered KYC (via Sumsub); global AML/CFT progra...,February 2025 Lazarus Group breach (~US $1.4B–...,https://www.coindesk.com/policy/2025/05/29/cry...,NaN,NaN,NaN
2,Isha,Coinbase Exchange,NaN,$420 billion in user traded assets,"USA, but no physical HQ","Publicly listed in NYSE, ~58% outside investors","Offers: Spot, futures(more features in the nea...",Spot: December 2023 Futures: Traditional ...,2024 Revenue: $6.6 billion 2024 Profit: $2...,"MiCA: June 20, 2025. ...","Bitlicense: January 2017, MiCA mentioned <---","Tax Disclosure, account verifying(counter terr...",$100 million to NYDFS due to lack of customer ...,"Coinbase website, Google Search, Government we...",NaN,6/26,NaN


In [12]:
# Ved's Task: Listed and BVI

def calculate_bvi_listed_score(df: pd.DataFrame) -> pd.DataFrame:
	"""Compute columns for BVI and Listed using real column names.
	- BVI: derived from `Geo_BVI` if present, else 0
	- Listed: extracted from governance structure text
	"""
	result = df.copy()
	
	# BVI from one-hot geography (if exists)
	bvi_col = 'Geo_BVI'
	if bvi_col in result.columns:
		result['BVI'] = result[bvi_col].fillna(0).astype(int)
	else:
		result['BVI'] = 0
	
	# Ved's Listed extraction from governance structure
	gov_col = 'Ownership & Governance Structure'
	if gov_col in result.columns:
		listed_series = result[gov_col].astype(str).str.lower()
		listed_flags = listed_series.str.contains(
			r'listed|public|ipo|nasdaq|nyse|lse|tsx|publicly', 
			regex=True, na=False
		).astype(int)
		result['Listed'] = listed_flags
	else:
		result['Listed'] = 0
	
	return result

In [13]:
# Data quality profiling cell: summarize coverage and inferred fields

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

profile_df = profile_data(original_df)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [14]:
# Data quality profiling cell: summarize coverage and inferred fields

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

profile_df = profile_data(original_df)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [15]:
# Data quality profiling cell: summarize coverage and inferred fields

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

profile_df = profile_data(original_df)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [16]:
# Data quality profiling cell: summarize coverage and inferred fields

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

profile_df = profile_data(original_df)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [17]:
# Data quality profiling cell: summarize coverage and inferred fields

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

profile_df = profile_data(original_df)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [18]:
# Ethan's Task: License_Count using real column names

LICENSE_TEXT_COLS = [
	'Regulatory Exposure (licenses, jurisdictions)',
	'Key Regulatory Frameworks (MiCA, BitLicense, etc.)'
]

COMPLIANCE_TEXT_COLS = [
	'Compliance Requirements (AML/KYC, disclosure, etc.)'
]


def _normalize_license_tokens(text: str) -> set:
	if not isinstance(text, str) or not text.strip():
		return set()
	raw = [p.strip().lower() for p in re.split(r'[;,/\n\|]', text) if p.strip()]
	canon = set()
	for token in raw:
		for k, v in LICENSE_CANON_MAP.items():
			if k in token:
				canon.add(v)
				break
		else:
			canon.add(token)
	return canon


def _count_distinct_licenses(text: str) -> int:
	return len(_normalize_license_tokens(text))


def calculate_license_score(df: pd.DataFrame) -> pd.DataFrame:
	"""Count distinct licenses/registrations using canonical mapping from real columns."""
	result = df.copy()
	canon_sets = [set() for _ in range(len(result))]
	
	# Parse from actual text columns in the spreadsheet
	for col in LICENSE_TEXT_COLS:
		if col in result.columns:
			series_sets = result[col].astype(str).apply(_normalize_license_tokens)
			canon_sets = [a.union(b) for a, b in zip(canon_sets, series_sets)]
	
	# Also check for any explicit one-hot regulatory flags if present
	one_hot_reg_cols = [
		'Reg_MiCA','Reg_BitLicense','Reg_FCA','Reg_FinCEN_MSB','Reg_VASP',
		'Reg_FATF','Reg_AUSTRAC','Reg_FINTRAC','Reg_SEC_CFTC'
	]
	available_reg_cols = [c for c in one_hot_reg_cols if c in result.columns]
	if available_reg_cols:
		for idx in result.index:
			for c in available_reg_cols:
				if pd.to_numeric(result.at[idx, c], errors='coerce') == 1:
					key = c.replace('Reg_', '')
					canon_sets[idx].add(LICENSE_CANON_MAP.get(key.lower(), key))
	
	result['License_Count'] = pd.Series([len(s) for s in canon_sets], index=result.index).astype(int)
	return result


import re

def _kyc_strength(text: str) -> int:
	if not isinstance(text, str):
		return 0
	t = text.lower()
	if 'full kyc' in t or 'mandatory kyc' in t:
		return 3
	if 'tiered kyc' in t:
		return 2
	if 'optional kyc' in t or 'partial kyc' in t:
		return 1
	if 'kyc' in t:  # Any mention of KYC gets at least 1 point
		return 1
	return 0


def _por_strength(text: str) -> int:
	if not isinstance(text, str):
		return 0
	t = text.lower()
	keywords = ['proof of reserves','attest','audit','audited','merkle','reserves']
	return int(any(k in t for k in keywords))


def calculate_compliance_maturity(df: pd.DataFrame) -> pd.DataFrame:
	"""Parse compliance maturity from real spreadsheet columns."""
	result = df.copy()
	
	# Use the actual compliance column from the spreadsheet
	comp_col = 'Compliance Requirements (AML/KYC, disclosure, etc.)'
	
	if comp_col in result.columns:
		comp_text = result[comp_col].astype(str)
		
		# Parse KYC subcategories
		result['KYC_Full'] = comp_text.apply(lambda x: int('full kyc' in x.lower() or 'mandatory kyc' in x.lower()))
		result['KYC_Tiered'] = comp_text.apply(lambda x: int('tiered kyc' in x.lower()))
		result['KYC_Optional'] = comp_text.apply(lambda x: int('optional kyc' in x.lower() or 'partial kyc' in x.lower()))
		result['Proof_of_Reserves'] = comp_text.apply(_por_strength)
		result['KYC_Strength_Score'] = comp_text.apply(_kyc_strength)
	else:
		# Fallback if column doesn't exist
		result['KYC_Full'] = 0
		result['KYC_Tiered'] = 0
		result['KYC_Optional'] = 0
		result['Proof_of_Reserves'] = 0
		result['KYC_Strength_Score'] = 0
	
	result['Compliance_Maturity'] = result['KYC_Strength_Score'] + result['Proof_of_Reserves']
	return result

In [19]:
# Adhiraj's Task: Incident Count and Country Regulation using real column names

INCIDENT_TEXT_COLS = [
	'Regulatory Incidents (fines, violations, etc.)'
]

HQ_COUNTRY_COL = 'Country/Region(s) of Operation'


def _count_incidents(text: str) -> int:
	if not isinstance(text, str) or not text.strip():
		return 0
	# Count incidents by splitting on common delimiters
	parts = [p.strip() for p in re.split(r'[;\n\|]', text) if p.strip()]
	return len(parts)


def _incident_severity_score(text: str) -> float:
	if not isinstance(text, str) or not text.strip():
		return 0.0
	lower = text.lower()
	score = 0.0
	for pattern, weight in INCIDENT_SEVERITY_WEIGHTS:
		if re.search(pattern, lower):
			score = max(score, weight)
	return score


def calculate_incident_score(df: pd.DataFrame) -> pd.DataFrame:
	result = df.copy()
	inc_counts = pd.Series(0, index=result.index)
	inc_severity = pd.Series(0.0, index=result.index)
	
	# Parse from the actual incident column in the spreadsheet
	for col in INCIDENT_TEXT_COLS:
		if col in result.columns:
			vals = result[col].astype(str)
			inc_counts = np.maximum(inc_counts.values, vals.apply(_count_incidents).values)
			inc_severity = np.maximum(inc_severity.values, vals.apply(_incident_severity_score).values)
	
	# Also check for any explicit incident flags if present
	flag_cols = [c for c in result.columns if isinstance(c, str) and c.lower().startswith('incident_')]
	if flag_cols:
		inc_counts = np.maximum(inc_counts, result[flag_cols].sum(axis=1))
	
	result['Incident_Count'] = inc_counts.astype(int)
	result['Incident_Severity'] = inc_severity
	return result


def _extract_primary_country(country_text: str) -> str:
	"""Extract primary country from country/region text."""
	if not isinstance(country_text, str) or not country_text.strip():
		return 'Unknown'
	
	# Clean and extract first country mentioned
	text = country_text.strip()
	
	# Handle common patterns
	if 'global' in text.lower() or 'worldwide' in text.lower():
		# Try to extract HQ info
		if 'hq' in text.lower() or 'headquarter' in text.lower():
			# Extract what comes after HQ
			parts = re.split(r'hq|headquarter', text.lower())
			if len(parts) > 1:
				hq_part = parts[1].strip()
				# Extract country name from parentheses or after "in"
				if '(' in hq_part:
					country = hq_part.split('(')[1].split(')')[0].strip()
				elif ' in ' in hq_part:
					country = hq_part.split(' in ')[1].split(',')[0].strip()
				else:
					country = hq_part.split(',')[0].strip()
				return country.title()
	
	# Split by common delimiters and take first
	first_part = re.split(r'[,;&]', text)[0].strip()
	
	# Clean up common patterns
	first_part = re.sub(r'^primarily\s+in\s+', '', first_part, flags=re.IGNORECASE)
	first_part = re.sub(r'^\s*but\s+', '', first_part, flags=re.IGNORECASE)
	
	return first_part.title()


def calculate_country_regulation(df: pd.DataFrame) -> pd.DataFrame:
	"""Build a per-country baseline using licensing and compliance maturity priors."""
	result = df.copy()
	
	if HQ_COUNTRY_COL not in result.columns:
		result['Country_Reg'] = 0.0
		return result
	
	# Require License_Count and Compliance_Maturity computed
	for needed in ['License_Count','Compliance_Maturity']:
		if needed not in result.columns:
			result[needed] = 0
	
	# Extract primary country for each exchange
	result['Primary_Country'] = result[HQ_COUNTRY_COL].apply(_extract_primary_country)
	
	# Calculate country-level statistics
	country_stats = (
		result.groupby('Primary_Country')[['License_Count','Compliance_Maturity']]
		.mean()
		.rename(columns={
			'License_Count': 'Country_License_Mean',
			'Compliance_Maturity': 'Country_Compliance_Mean'
		})
	)
	
	# Merge back to main dataframe
	result = result.merge(
		country_stats,
		left_on='Primary_Country',
		right_index=True,
		how='left'
	)
	
	result[['Country_License_Mean','Country_Compliance_Mean']] = result[['Country_License_Mean','Country_Compliance_Mean']].fillna(0)
	
	# Simple composite score
	result['Country_Reg'] = (
		0.5 * result['Country_License_Mean'] + 0.5 * result['Country_Compliance_Mean']
	)
	
	return result

In [20]:
# Product correlation placeholder (Adhiraj):
# Build scalar risks for product complexity to later use as independent variable.
PRODUCT_COLS = ['Product_Options','Product_Futures','Product_Margin','Product_Staking','Product_P2P_OTC','Product_NFTs','Product_Launchpad','Product_Crypto_Loans','Product_Spot']

PRODUCT_WEIGHTS = {
	'Product_Options': 3.0,
	'Product_Futures': 3.0,
	'Product_Margin': 2.0,
	'Product_Staking': 1.0,
	'Product_P2P_OTC': 1.0,
	'Product_NFTs': 0.5,
	'Product_Launchpad': 1.0,
	'Product_Crypto_Loans': 2.0,
	'Product_Spot': 0.5,
}


def calculate_product_complexity(df: pd.DataFrame) -> pd.DataFrame:
	result = df.copy()
	available = [c for c in PRODUCT_COLS if c in result.columns]
	if not available:
		result['Product_Complexity'] = 0.0
		return result
	weights = {k: PRODUCT_WEIGHTS[k] for k in available}
	result['Product_Complexity'] = (result[available] * pd.Series(weights)).sum(axis=1)
	result['Num_Products'] = result[available].sum(axis=1)
	return result


In [21]:
from collections import defaultdict
import re as _re

# Use the actual column names from the spreadsheet
PRODUCT_TEXT_COLS = [
	'Products Offered (spot, futures, options, etc.)',
	'Product Launch Dates',
	'Link to doc with full analysis'
]

PRODUCT_KEYWORDS = {
	'Product_Options': [r'option(s)?', r'perp(etuals)? option', r'vanilla option'],
	'Product_Futures': [r'future(s)?', r'perp(etu)?als?', r'perpetual'],
	'Product_Margin': [r'margin', r'leverage(d)? spot', r'leverage'],
	'Product_Staking': [r'staking', r'stake\b', r'earn(\b|ing|s)?', r'savings'],
	'Product_P2P_OTC': [r'p2p', r'otc', r'peer[- ]to[- ]peer'],
	'Product_NFTs': [r'nft(s)?', r'non[- ]fungible'],
	'Product_Launchpad': [r'launchpad', r'ieo', r'ido'],
	'Product_Crypto_Loans': [r'loan(s)?', r'borrow', r'lending'],
	'Product_Spot': [r'spot(\b| market| trading)'],
}


def _row_text_concat(row: pd.Series) -> str:
	parts = []
	for c in PRODUCT_TEXT_COLS:
		if c in row and isinstance(row[c], str) and row[c] != 'nan':
			parts.append(row[c])
	return ' | '.join(parts).lower()


def infer_product_flags(df: pd.DataFrame) -> pd.DataFrame:
	result = df.copy()
	text_series = result.apply(_row_text_concat, axis=1)

	for col, patterns in PRODUCT_KEYWORDS.items():
		regex = _re.compile('|'.join(patterns), flags=_re.IGNORECASE)
		inferred = text_series.apply(lambda t: int(bool(regex.search(t))))
		result[f'{col}_inferred'] = inferred
		
		# If explicit column exists, combine with inferred
		if col in result.columns and result[col].dropna().shape[0] > 0:
			explicit = pd.to_numeric(result[col], errors='coerce').fillna(0).astype(int)
			final = np.maximum(explicit, inferred)
		else:
			final = inferred
		result[f'{col}_final'] = final

	final_cols = [f'{c}_final' for c in PRODUCT_KEYWORDS.keys() if f'{c}_final' in result.columns]
	unknown_mask = (result[final_cols].sum(axis=1) == 0) if final_cols else pd.Series(True, index=result.index)
	result['Product_Inference_Unknown'] = unknown_mask.astype(int)
	return result


def calculate_product_complexity_nlp(df: pd.DataFrame) -> pd.DataFrame:
	"""Compute product complexity using 'final' flags after NLP inference."""
	result = infer_product_flags(df)
	final_cols = [f'{c}_final' for c in PRODUCT_KEYWORDS.keys() if f'{c}_final' in result.columns]
	
	if not final_cols:
		result['Product_Complexity'] = 0.0
		result['Num_Products'] = 0.0
		return result
	
	weights = {f'{k}_final': w for k, w in PRODUCT_WEIGHTS.items() if f'{k}_final' in result.columns}
	result['Product_Complexity'] = (result[list(weights.keys())] * pd.Series(weights)).sum(axis=1)
	result['Num_Products'] = result[final_cols].sum(axis=1)
	return result


def sample_uncertain_products(df: pd.DataFrame, n: int = 25) -> pd.DataFrame:
	aug = infer_product_flags(df)
	uncertain = aug[aug['Product_Inference_Unknown'] == 1]
	return uncertain.sample(min(n, len(uncertain)), random_state=42) if len(uncertain) else uncertain

In [22]:
# Assembly: main execution to compute Exchange_Reg

def assemble_exchange_reg(df: pd.DataFrame) -> pd.DataFrame:
	step = df.copy()
	step = calculate_bvi_listed_score(step)
	step = calculate_license_score(step)
	step = calculate_compliance_maturity(step)
	step = calculate_incident_score(step)
	step = calculate_country_regulation(step)
	step = calculate_product_complexity_nlp(step)
	listed_weight = 2.0
	listed_effect = listed_weight * step['Listed'].fillna(0)
	step['Exchange_Reg'] = (
		listed_effect
		+ step['License_Count'].fillna(0)
		+ step['Incident_Count'].fillna(0)
		+ step['Compliance_Maturity'].fillna(0)
		+ step['Country_Reg'].fillna(0)
		- step['BVI'].fillna(0)
	)
	return step

scored_df = assemble_exchange_reg(original_df)
scored_df[['Crypto Exchange','Exchange_Reg']].head(10)


,Crypto Exchange,Exchange_Reg
0,Binance,18.166667
1,ByBit,22.500000
2,Coinbase Exchange,10.500000
3,Upbit,23.000000
4,OKX,30.000000
5,Bitget,21.500000
6,MEXC,13.000000
7,Gate,34.000000
8,HTX,13.500000
9,KuCoin,20.333333


In [23]:
# Regression Model: relate Exchange_Reg with product complexity and geography/regulatory flags

model_df = scored_df.copy()

geo_cols = [c for c in model_df.columns if isinstance(c, str) and c.startswith('Geo_')]
reg_cols = ['Reg_MiCA','Reg_BitLicense','Reg_FCA','Reg_FinCEN_MSB','Reg_VASP','Reg_FATF','Reg_AUSTRAC','Reg_FINTRAC','Reg_SEC_CFTC']
reg_cols = [c for c in reg_cols if c in model_df.columns]

feature_cols = ['Product_Complexity','Num_Products'] + geo_cols + reg_cols
available_features = [c for c in feature_cols if c in model_df.columns]

model_df = model_df.dropna(subset=['Exchange_Reg'])

if available_features:
	formula = 'Exchange_Reg ~ ' + ' + '.join([f'Q("{c}")' for c in available_features])
	try:
		ols_model = smf.ols(formula=formula, data=model_df).fit()
		print(ols_model.summary())
	except Exception as e:
		print('Regression failed:', e)
else:
	print('No features available for regression yet.')


                            OLS Regression Results                            
Dep. Variable:           Exchange_Reg   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     22.93
Date:                Wed, 13 Aug 2025   Prob (F-statistic):           7.27e-10
Time:                        17:07:17   Log-Likelihood:                -754.06
No. Observations:                 251   AIC:                             1514.
Df Residuals:                     248   BIC:                             1525.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [24]:
# Data quality profiling cell: summarize coverage and inferred fields (robust)

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

# Ensure we have a dataframe to profile
try:
	_df_for_profile = original_df
except NameError:
	try:
		_df_for_profile = load_original_df()
	except Exception:
		_df_for_profile = pd.read_csv(ORIGINAL_CSV_PATH)

profile_df = profile_data(_df_for_profile)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [25]:
# Assembly: main execution to compute Exchange_Reg with exact formula

def assemble_exchange_reg(df: pd.DataFrame) -> pd.DataFrame:
	step = df.copy()
	
	# Run all the component calculations
	step = calculate_bvi_listed_score(step)
	step = calculate_license_score(step)
	step = calculate_compliance_maturity(step)
	step = calculate_incident_score(step)
	step = calculate_country_regulation(step)
	step = calculate_product_complexity_nlp(step)
	
	# Exact formula from meeting notes: Exchange_Reg = (Listed) + License_Count + Incident Count + Compliance_Maturity + Country_Reg − BVI
	# Listed is overweighted (importance factor)
	listed_weight = 2.0  # Overweight Listed for transparency
	
	step['Exchange_Reg'] = (
		listed_weight * step['Listed'].fillna(0)
		+ step['License_Count'].fillna(0)
		+ step['Incident_Count'].fillna(0)
		+ step['Compliance_Maturity'].fillna(0)
		+ step['Country_Reg'].fillna(0)
		- step['BVI'].fillna(0)
	)
	
	return step

# Execute the assembly
scored_df = assemble_exchange_reg(original_df)

# Show sample results with key columns
print("Sample Exchange_Reg results:")
sample_cols = ['Crypto Exchange', 'Listed', 'License_Count', 'Incident_Count', 'Compliance_Maturity', 'Country_Reg', 'BVI', 'Exchange_Reg']
available_cols = [c for c in sample_cols if c in scored_df.columns]
print(scored_df[available_cols].head(10).to_string())

# Check if we're getting non-zero values for components
print(f"\nNon-zero rates:")
for col in ['License_Count', 'Compliance_Maturity', 'Product_Complexity', 'Listed']:
    if col in scored_df.columns:
        non_zero_rate = (scored_df[col] > 0).mean()
        print(f"{col}: {non_zero_rate:.2%}")

print(f"\nExchange_Reg equals Incident_Count only: {(scored_df['Exchange_Reg'] == scored_df['Incident_Count']).mean():.2%}")

Sample Exchange_Reg results:
     Crypto Exchange  Listed  License_Count  Incident_Count  Compliance_Maturity  Country_Reg  BVI  Exchange_Reg
0            Binance       0             12               1                    1     4.166667    0     18.166667
1              ByBit       0              8               6                    3     5.500000    0     22.500000
2  Coinbase Exchange       1              5               1                    0     2.500000    0     10.500000
3              Upbit       1              9               6                    1     5.000000    0     23.000000
4                OKX       0             16               3                    2     9.000000    0     30.000000
5             Bitget       0             10               5                    1     5.500000    0     21.500000
6               MEXC       1              3               2                    3     3.000000    0     13.000000
7               Gate       0             18               1        

In [26]:
# Improved Regression Model: relate Exchange_Reg with meaningful features

model_df = scored_df.copy()

# Drop rows with missing Exchange_Reg
model_df = model_df.dropna(subset=['Exchange_Reg'])

# Potential geographic and regulatory features
geo_cols = [c for c in model_df.columns if isinstance(c, str) and c.startswith('Geo_')]
reg_cols = ['Reg_MiCA','Reg_BitLicense','Reg_FCA','Reg_FinCEN_MSB','Reg_VASP','Reg_FATF','Reg_AUSTRAC','Reg_FINTRAC','Reg_SEC_CFTC']
reg_cols = [c for c in reg_cols if c in model_df.columns]

# Primary features to include in regression
primary_features = ['Product_Complexity', 'Num_Products', 'Incident_Severity']
available_primary = [c for c in primary_features if c in model_df.columns and model_df[c].var() > 0]

# Combine all potential features
feature_cols = available_primary + geo_cols + reg_cols
available_features = [c for c in feature_cols if c in model_df.columns and model_df[c].var() > 0]

print(f"Available features for regression: {available_features}")

if available_features:
    try:
        # Create formula
        formula = 'Exchange_Reg ~ ' + ' + '.join([f'Q("{c}")' for c in available_features])
        print(f"Regression formula: {formula}")
        
        # Fit model
        ols_model = smf.ols(formula=formula, data=model_df).fit()
        print(ols_model.summary())
        
        # Show some correlation analysis
        print(f"\nCorrelations with Exchange_Reg:")
        for feat in available_features:
            corr = model_df[['Exchange_Reg', feat]].corr().iloc[0,1]
            print(f"{feat}: {corr:.3f}")
            
    except Exception as e:
        print(f'Regression failed: {e}')
        print("Trying simple correlations instead...")
        for feat in available_features:
            try:
                corr = model_df[['Exchange_Reg', feat]].corr().iloc[0,1]
                print(f"{feat}: {corr:.3f}")
            except:
                pass
else:
    print('No features with variance available for regression yet.')
    print("This suggests the parsing may need refinement.")
    print(f"Unique Exchange_Reg values: {sorted(model_df['Exchange_Reg'].unique())}")

Available features for regression: ['Product_Complexity', 'Num_Products', 'Incident_Severity']
Regression formula: Exchange_Reg ~ Q("Product_Complexity") + Q("Num_Products") + Q("Incident_Severity")
                            OLS Regression Results                            
Dep. Variable:           Exchange_Reg   R-squared:                       0.244
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     26.64
Date:                Wed, 13 Aug 2025   Prob (F-statistic):           5.78e-15
Time:                        17:07:17   Log-Likelihood:                -740.17
No. Observations:                 251   AIC:                             1488.
Df Residuals:                     247   BIC:                             1502.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
           

In [27]:
# Data quality profiling cell: summarize coverage and inferred fields (robust)

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

# Ensure we have a dataframe to profile
try:
	_df_for_profile = original_df
except NameError:
	try:
		_df_for_profile = load_original_df()
	except Exception:
		_df_for_profile = pd.read_csv(ORIGINAL_CSV_PATH)

profile_df = profile_data(_df_for_profile)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [28]:
# Save central master CSV to parent directory (top-level)
from pathlib import Path

# Save to parent directory to avoid nested folder issues
parent_dir = Path.cwd().parent if 'stat-analysis' in str(Path.cwd()) else Path.cwd()
MASTER_CSV_PATH = parent_dir / 'central_master.csv'

cols_preferred_order = [
	'Crypto Exchange','Listed','License_Count','Incident_Count','Incident_Severity',
	'KYC_Full','KYC_Tiered','KYC_Optional','Proof_of_Reserves','KYC_Strength_Score',
	'Compliance_Maturity','Country_Reg','Primary_Country','BVI','Product_Complexity',
	'Num_Products','Exchange_Reg'
]

existing_cols = [c for c in cols_preferred_order if c in scored_df.columns]
master_cols = existing_cols + [c for c in scored_df.columns if c not in existing_cols]

scored_df[master_cols].to_csv(MASTER_CSV_PATH, index=False)
print(f'Saved: {MASTER_CSV_PATH} with shape {scored_df.shape}')
print(f'Key columns included: {existing_cols}')

Saved: C:\Users\achom\OneDrive\Documents\assip\central_master.csv with shape (251, 54)
Key columns included: ['Crypto Exchange', 'Listed', 'License_Count', 'Incident_Count', 'Incident_Severity', 'KYC_Full', 'KYC_Tiered', 'KYC_Optional', 'Proof_of_Reserves', 'KYC_Strength_Score', 'Compliance_Maturity', 'Country_Reg', 'Primary_Country', 'BVI', 'Product_Complexity', 'Num_Products', 'Exchange_Reg']


In [29]:
# Data quality profiling cell: summarize coverage and inferred fields (robust)

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

# Ensure we have a dataframe to profile
try:
	_df_for_profile = original_df
except NameError:
	try:
		_df_for_profile = load_original_df()
	except Exception:
		_df_for_profile = pd.read_csv(ORIGINAL_CSV_PATH)

profile_df = profile_data(_df_for_profile)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155


In [30]:
# Data quality profiling cell: summarize coverage and inferred fields (robust)

def profile_data(df: pd.DataFrame) -> pd.DataFrame:
	cols = df.columns
	summary = []
	for c in cols:
		series = df[c]
		missing = series.isna().mean()
		unique = series.nunique(dropna=True)
		dtype = str(series.dtype)
		summary.append({'column': c, 'dtype': dtype, 'missing_pct': round(missing, 4), 'unique': int(unique)})
	return pd.DataFrame(summary).sort_values(['missing_pct','column'])

# Ensure we have a dataframe to profile
try:
	_df_for_profile = original_df
except NameError:
	try:
		_df_for_profile = load_original_df()
	except Exception:
		_df_for_profile = pd.read_csv(ORIGINAL_CSV_PATH)

profile_df = profile_data(_df_for_profile)
profile_df.head(25)


,column,dtype,missing_pct,unique
0,`,object,0.0000,10
1,Crypto Exchange,object,0.0000,251
4,Country/Region(s) of Operation,object,0.0040,247
3,Exchange Size (Large/Medium/Small),object,0.0080,233
6,"Products Offered (spot, futures, options, etc.)",object,0.0120,241
5,Ownership & Governance Structure,object,0.0159,245
11,"Compliance Requirements (AML/KYC, disclosure, ...",object,0.0239,244
7,Product Launch Dates,object,0.0239,221
9,"Regulatory Exposure (licenses, jurisdictions)",object,0.0319,242
8,Revenue/Profit by Region,object,0.0478,155
